In [1]:
import pandas as pd
import surprise as sp
from itertools import combinations
from collections import Counter
import ast 
from sklearn.preprocessing import MultiLabelBinarizer
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import numpy as np
from collections import defaultdict

In [2]:
import os
new_dir = '/Users/smash/Documents/GitHub/GMR'  # Replace with the actual path
os.chdir(new_dir)
print(os.getcwd())

/Users/smash/Documents/GitHub/GMR


In [3]:
df = pd.read_csv('rawdata/steam/purchased_games.csv')
print(df.head())
games_df = pd.read_csv('rawdata/steam/games.csv')
print(games_df.shape)
games_df = games_df[games_df["supported_languages"].str.contains("English", case=False, na=True)]
print(games_df.shape)
games_df = games_df[['gameid','title']]
#games_df['gameid'] = games_df['gameid'].astype(str) 
#df = df[:100]
df = df.dropna()
df['library'] = df['library'].apply(ast.literal_eval)

            playerid                                            library
0  76561198060698936  [60, 1670, 3830, 1600, 2900, 2910, 2920, 4800,...
1  76561198287452552  [10, 80, 100, 240, 2990, 6880, 6910, 6920, 698...
2  76561198040436563  [10, 80, 100, 300, 20, 30, 40, 50, 60, 70, 130...
3  76561198042412488  [300, 240, 220, 320, 360, 4300, 4800, 4000, 61...
4  76561198119605821  [47870, 108600, 550, 271590, 331470, 381210, 2...
(98248, 7)
(94260, 7)


In [4]:
games_dict = games_df.set_index('gameid')['title'].to_dict()

In [11]:
mlb = MultiLabelBinarizer()
binary_matrix = mlb.fit_transform(df['library'])
game_columns = mlb.classes_
df_binary = pd.DataFrame(binary_matrix, columns=game_columns, index=df.index)
df_expanded = pd.concat([df[['playerid']], df_binary], axis=1)
cols_to_drop = (df == 0).all()
cols_to_drop = cols_to_drop[cols_to_drop].index.tolist()
df_expanded = df_expanded.drop(columns=cols_to_drop)
df_expanded = df_expanded.rename(columns=games_dict)
columns_to_drop = [col for col in df_expanded.columns if str(col).isdigit()]
df_expanded = df_expanded.drop(columns=columns_to_drop)


In [12]:
rows_to_drop = (df_expanded == 0).all(axis=1)
df_expanded = df_expanded[~rows_to_drop]


In [13]:
df_expanded.head()

,playerid,Counter-Strike,Team Fortress Classic,Day of Defeat,Deathmatch Classic,Half-Life: Opposing Force,Ricochet,Half-Life,Counter-Strike: Condition Zero,Counter-Strike: Condition Zero,...,Sexy Memory Puzzle - Futanari BDSM,College Gay Sex - Episode 8,Sexy Memory Puzzle - Gay Affair,Futanari Sex Adventures - Episode 6,Sexy Memory Puzzle - Dildo Girl,Harem Inn,They're Alive!,Utopia City,Rat Hunter,Metathrone
0,76561198060698936,0,1,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,76561198287452552,1,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,76561198040436563,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,76561198042412488,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,76561198119605821,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
print(df_expanded.shape)


(46941, 36314)


In [15]:
df_exploded = df.explode('library')
df_exploded = df_exploded.rename(columns={'playerid': 'user', 'library': 'item'})
df_exploded['rating'] = 1.0
#df_exploded['item'] = df_exploded['item'].astype(int)
df_exploded['item'] = df_exploded['item'].map(games_dict)
df_surprise = df_exploded[['user', 'item', 'rating']].drop_duplicates()
df_surprise = df_surprise.dropna()
print(df_surprise.shape)
print(df_surprise.head())

(9994427, 3)
                user                                 item  rating
0  76561198060698936                             Ricochet     1.0
0  76561198060698936  Iron Warriors: T - 72 Tank Command      1.0
0  76561198060698936                          Psychonauts     1.0
0  76561198060698936                     Dangerous Waters     1.0
0  76561198060698936                 688(I) Hunter/Killer     1.0


In [16]:
reader = Reader(rating_scale=(0, 1)) 
svd_data = Dataset.load_from_df(df_surprise[['user', 'item', 'rating']], reader)
trainset, testset = train_test_split(svd_data, test_size=0.25, random_state=42)
svd_model = SVD()
svd_model.fit(trainset)


In [24]:
target_user_id = 76561198119605821
all_game_ids = df_surprise['item'].unique()

In [25]:
games_owned_by_target = df_surprise[df_surprise['user'] == target_user_id]['item'].tolist()

print(f"Games owned by {target_user_id}: {games_owned_by_target}")

Games owned by 76561198119605821: ['Need For Speed: Hot Pursuit', 'Project Zomboid', 'Left 4 Dead 2', 'Grand Theft Auto V', 'Everlasting Summer', 'Dead by Daylight', 'Brawlhalla', 'World of Tanks Blitz', 'Call of Duty®: WWII', 'War Robots', 'The Crew™ 2', 'Counter-Strike 2', 'Drink More Glurp', 'Warface: Clutch', 'Crab Game', 'MEGA MAN X DiVE Offline', 'TEKKEN 8', 'NARUTO X BORUTO Ultimate Ninja STORM CONNECTIONS', 'WWE 2K24']


In [26]:
games_to_predict = [game_id for game_id in all_game_ids if game_id not in games_owned_by_target]

In [27]:
predictions = []
for game_id in games_to_predict:
    # Predict rating for user-item pair
    # uid=target_user_id, iid=game_id
    # The 'r_ui=1' is a placeholder true rating, it doesn't affect SVD prediction score itself
    pred = svd_model.predict(uid=target_user_id, iid=game_id, verbose=False) 
    predictions.append(pred)

In [31]:
predictions.sort(key=lambda x: x.est, reverse=True)

n_recommendations = 10

top_n_recommendations = predictions[:n_recommendations]

print(f"\nTop {n_recommendations} game recommendations for {target_user_id}:")
for pred in top_n_recommendations:
    if str(pred.iid) in games_dict:
      game_name = games_dict[pred.iid]
    else:
        game_name = "error"
    print(f"- Game: {pred.iid}, Predicted Score: {pred.est:.4f}")


Top 10 game recommendations for 76561198119605821:
- Game: Ricochet, Predicted Score: 1.0000
- Game: Iron Warriors: T - 72 Tank Command , Predicted Score: 1.0000
- Game: Psychonauts, Predicted Score: 1.0000
- Game: 688(I) Hunter/Killer, Predicted Score: 1.0000
- Game: Garry's Mod, Predicted Score: 1.0000
- Game: Hitman: Codename 47, Predicted Score: 1.0000
- Game: Hitman 2: Silent Assassin, Predicted Score: 1.0000
- Game: Hitman: Blood Money, Predicted Score: 1.0000
- Game: Space Empires IV Deluxe, Predicted Score: 1.0000
- Game: Space Empires V, Predicted Score: 1.0000
